In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv('data/cleaned_dataset.csv')

In [3]:
df.head()

,Gesture,Ax,Ay,Az,Gx,Gy,Gz
0,0,0.327250,0.972020,0.978133,0.972500,0.282577,0.551446
1,0,0.323264,0.978824,0.978574,0.968821,0.297214,0.549468
2,0,0.331779,0.970411,0.978970,0.970301,0.260290,0.551673
3,0,0.313128,0.981825,0.978850,0.968926,0.284897,0.547513
4,0,0.222645,0.968425,0.979011,0.969466,0.267591,0.550981


In [4]:
X = df[['Ax', 'Ay', 'Az', 'Gx', 'Gy', 'Gz']]
y = df['Gesture']

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15)

In [6]:
y_train.head()

206    1
56     0
27     0
186    1
169    1
Name: Gesture, dtype: int64

In [7]:
X_train.head()

,Ax,Ay,Az,Gx,Gy,Gz
206,0.456646,0.853639,0.978528,0.969101,0.274786,0.520502
56,0.330482,0.920396,0.979440,0.968450,0.255034,0.551264
27,0.403998,0.950270,0.974576,0.967800,0.257439,0.551752
186,0.293535,0.914164,0.978645,0.917306,0.262041,0.554830
169,0.322265,0.925372,0.978472,0.969835,0.242481,0.554506


In [8]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
import tensorflow as tf

In [9]:
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_schedule = LearningRateScheduler(lambda epoch: 1e-3 * 0.9 ** epoch)

In [11]:
history = model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test),
                    callbacks=[early_stopping, lr_schedule])

Epoch 1/200
6/6 [==============================] - 2s 53ms/step - loss: 0.8795 - accuracy: 0.4641 - val_loss: 0.6570 - val_accuracy: 0.6667 - lr: 0.0010
Epoch 2/200
6/6 [==============================] - 0s 10ms/step - loss: 0.7520 - accuracy: 0.5414 - val_loss: 0.6450 - val_accuracy: 0.6667 - lr: 9.0000e-04
Epoch 3/200
6/6 [==============================] - 0s 11ms/step - loss: 0.7217 - accuracy: 0.5912 - val_loss: 0.6378 - val_accuracy: 0.6970 - lr: 8.1000e-04
Epoch 4/200
6/6 [==============================] - 0s 12ms/step - loss: 0.6660 - accuracy: 0.7238 - val_loss: 0.6307 - val_accuracy: 0.6970 - lr: 7.2900e-04
Epoch 5/200
6/6 [==============================] - 0s 12ms/step - loss: 0.6170 - accuracy: 0.7017 - val_loss: 0.6249 - val_accuracy: 0.6970 - lr: 6.5610e-04
Epoch 6/200
6/6 [==============================] - 0s 13ms/step - loss: 0.5629 - accuracy: 0.7348 - val_loss: 0.6201 - val_accuracy: 0.7273 - lr: 5.9049e-04
Epoch 7/200
6/6 [==============================] - 0s 15ms/ste

In [12]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

2/2 [==============================] - 0s 0s/step - loss: 0.4661 - accuracy: 0.8788
Test Loss: 0.46606600284576416, Test Accuracy: 0.8787878751754761


In [13]:
y_hat = model.predict(X_test)
y_hat = [0 if val < 0.5 else 1 for val in y_hat]

2/2 [==============================] - 0s 0s/step


In [14]:
print(y_train.value_counts())

Gesture
1    94
0    87
Name: count, dtype: int64


In [15]:
accuracy_score(y_test, y_hat)

0.8787878787878788

In [16]:
model.save('tfmodel.keras')

In [17]:
del model

In [18]:
model = load_model('tfmodel.keras')

In [19]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

2/2 [==============================] - 0s 6ms/step - loss: 0.4661 - accuracy: 0.8788
Test Loss: 0.4661
Test Accuracy: 87.88%


In [20]:
# MasoudKaviani.ir